# Processamento de texto

Iremos estudar brevemente como processar texto usando redes neurais. Como esse não é um objetivo do curso, utilizaremos a biblioteca Hugginface para fazer essa tarefa. Mas é importante entendermos o processamento realizado pela biblioteca.

### Hugginface

Primeiramente utilizaremos um pipeline do Hugginface, que é a interface de mais alto nível da biblioteca

Lista de modelos disponíveis para o pipeline: https://huggingface.co/models \
Lista de tarefas disponíveis para o pipeline: https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.pipeline.task

In [12]:
import torch
from torch import nn
from transformers import pipeline

pipe = pipeline(model='distilbert/distilbert-base-cased', task='feature-extraction', 
                return_tensors=True)

texts = ['Some sentence to test the model', 'Another sentence']

In [2]:
y = pipe(texts)
y

[tensor([[[ 0.3386,  0.0341, -0.0293,  ..., -0.2339,  0.0973,  0.0785],
          [ 0.1333, -0.3066,  0.1266,  ...,  0.0262,  0.0969,  0.2786],
          [ 0.2536, -0.0148,  0.1565,  ..., -0.0567, -0.1064,  0.1481],
          ...,
          [-0.0250, -0.2859,  0.0105,  ...,  0.2711, -0.1073,  0.3281],
          [ 0.0498, -0.1095, -0.2095,  ...,  0.3505, -0.0599, -0.0365],
          [ 0.7945,  0.0952, -0.0333,  ..., -0.0862,  0.5560, -0.2026]]]),
 tensor([[[ 0.3035,  0.0292, -0.0063,  ..., -0.0207,  0.2161,  0.0616],
          [-0.1794, -0.4330, -0.1119,  ..., -0.0574,  0.2387,  0.2266],
          [ 0.1669,  0.1894,  0.2134,  ...,  0.1244,  0.3058,  0.1806],
          [ 0.7503,  0.1748, -0.0862,  ...,  0.3358,  0.7509, -0.4554]]])]

In [3]:
y[0].shape

torch.Size([1, 8, 768])

Para cada sentença, é retornada uma matriz de tamanho 1x8x768. Vamos entender melhor essas dimensões mais abaixo.

O pipeline é composto por um tokenizador e uma rede neural:

In [4]:
tokenizer = pipe.tokenizer
model = pipe.model

print(tokenizer)
print(model)

DistilBertTokenizerFast(name_or_path='distilbert/distilbert-base-cased', vocab_size=28996, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(

In [5]:
tokens = tokenizer.encode(texts)
tokens

[101, 1789, 5650, 1106, 2774, 1103, 2235, 102, 2543, 5650, 102]

Cada sentença é dividida em tokens (input_ids), que representam elementos da sentença. 

In [6]:
for tok in tokens:
    print(tok, pipe.tokenizer.decode(tok))

101 [CLS]
1789 Some
5650 sentence
1106 to
2774 test
1103 the
2235 model
102 [SEP]
2543 Another
5650 sentence
102 [SEP]


A forma padrão de usar o tokenizador é chamando a instância da classe:

In [7]:
tokens = tokenizer(texts, return_tensors='pt', padding=True)
tokens

{'input_ids': tensor([[ 101, 1789, 5650, 1106, 2774, 1103, 2235,  102],
        [ 101, 2543, 5650,  102,    0,    0,    0,    0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0, 0, 0, 0]])}

`input_ids` são os tokens gerados para cada sentença. Sentenças curtas são preenchidas com 0 para ficarem com o mesmo tamanho que a maior sentença. 

O modelo BERT também possui a chamada `attention_mask`, que é uma máscara que indica quais tokens o modelo pode utilizar para fazer previsões. No nosso caso a máscara será usada somente para ignorar tokens de padding.

Para aplicar o modelo nos tokens, basta fazermos:

In [8]:
res = model(**tokens)
res[0].shape

torch.Size([2, 8, 768])

O modelo gera duas matrizes, uma para cada sentença. Cada sentença possui 8 tokens. Para cada token, são gerados 768 atributos. No caso do modelo BERT, é comum utilizar os atributos gerados para o token de classe (CLS) para caracterizar a sentença como um todo.

In [9]:
text_features = res[0][:,0]
text_features.shape

torch.Size([2, 768])

Utilização da GPU

In [ ]:
model.to('cuda')
# A tokenização é feita na CPU
tokens = tokenizer(texts, return_tensors='pt', padding=True)
# Envia os tensores gerados para a GPU
tokens = tokens.to('cuda')
# Aplica o modelo
res = model(**tokens)

### Codificador de texto

Vamos criar um modelo para codificar textos. Esse modelo será utilizado junto com um modelo de imagens.

In [11]:
from torch import nn
from transformers import pipeline

class TextEncoder(nn.Module):

    def __init__(self):
        super().__init__()

        # Carrega o pipeline do Hugginface, que inclui um tokenizador e
        # um modelo de classificação de texto
        pipe = pipeline(model='distilbert/distilbert-base-cased', task='feature-extraction')
        tokenizer = pipe.tokenizer
        model = pipe.model

        self.tokenizer = tokenizer
        self.model = model
        # id do token associado à classe
        self.cls_token_id = 0
        # Dimensão de saída do modelo distilbert
        self.feature_dim = 768

    def forward(self, text):

        # Se houver uma lista de textos, é preciso preencher com zeros
        # para deixá-los com mesmo tamanho
        padding = isinstance(text, (list, tuple))

        tokens = self.tokenizer(text, return_tensors='pt', padding=padding)

        # Envia o texto tokenizado para o mesmo device que o modelo
        tokens = tokens.to(self.model.device)
        res = self.model(**tokens)[0]

        # Acessa os atributos associados com o token de classe
        features = res[:, self.cls_token_id]
        
        return features

text_encoder = TextEncoder()
features = text_encoder(texts)
features.shape

torch.Size([2, 768])